In [1]:
import json
import joblib
import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
# Surface data requires GRIB1
set1_sfc = xr.open_dataset('data/set1.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'edition': 1}})
set2_sfc = xr.open_dataset('data/set2.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'edition': 1}})

sfc = xr.concat([set1_sfc, set2_sfc], dim='time')

In [3]:
# Model level data requires GRIB2
set1_mlev = xr.open_dataset('data/set1.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'edition': 2}})
set2_mlev = xr.open_dataset('data/set2.grib', engine='cfgrib', backend_kwargs={'filter_by_keys': {'edition': 2}})

# Adjust longitudes from 0-360 to -180 to 180 to be consistent with surface data
set1_mlev = set1_mlev.assign_coords(longitude=(((set1_mlev.longitude + 180) % 360) - 180))
set2_mlev = set2_mlev.assign_coords(longitude=(((set2_mlev.longitude + 180) % 360) - 180))

mlev = xr.concat([set1_mlev, set2_mlev], dim='time')

In [4]:
sfc

<xarray.Dataset> Size: 20MB
Dimensions:     (time: 8768, latitude: 9, longitude: 9)
Coordinates:
    number      int64 8B 0
  * time        (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
    step        timedelta64[ns] 8B 00:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 72B 36.0 35.25 34.5 33.75 ... 31.5 30.75 30.0
  * longitude   (longitude) float64 72B -86.0 -85.25 -84.5 ... -80.75 -80.0
    valid_time  (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
Data variables:
    u10         (time, latitude, longitude) float32 3MB 0.1425 0.06673 ... 7.265
    v10         (time, latitude, longitude) float32 3MB 3.888 3.758 ... 3.548
    d2m         (time, latitude, longitude) float32 3MB 290.4 289.8 ... 293.6
    t2m         (time, latitude, longitude) float32 3MB 292.2 291.3 ... 297.6
    sp          (time, latitude, longitude) float32 3MB 9.749e+04 ... 1.011e+05
    pm2p5       (time, latitude, longitude) float32 3MB 6.126e-09 ... 7.598e-09
    pm10        (time, latitude, longitude) float32 3MB 8.772e-09 ... 1.16e-08
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-17T19:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [5]:
mlev

<xarray.Dataset> Size: 34MB
Dimensions:     (time: 8768, latitude: 9, longitude: 9)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
    step        timedelta64[ns] 8B 00:00:00
    hybrid      float64 8B 60.0
  * latitude    (latitude) float64 72B 36.0 35.25 34.5 33.75 ... 31.5 30.75 30.0
    valid_time  (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
  * longitude   (longitude) float64 72B -86.0 -85.25 -84.5 ... -80.75 -80.0
Data variables:
    c2h6        (time, latitude, longitude) float32 3MB 4.884e-10 ... 3.732e-10
    oh          (time, latitude, longitude) float32 3MB 2.922e-15 ... 9.231e-14
    c5h8        (time, latitude, longitude) float32 3MB 1.819e-10 ... 9.002e-12
    hno3        (time, latitude, longitude) float32 3MB 3.915e-10 ... 2.14e-09
    no2         (time, latitude, longitude) float32 3MB 5.96e-09 ... 1.173e-09
    no          (time, latitude, longitude) float32 3MB 4.903e-11 ... 1.451e-10
    go3         (time, latitude, longitude) float32 3MB 4.276e-08 ... 6.73e-08
    c3h8        (time, latitude, longitude) float32 3MB 2.424e-10 ... 1.429e-10
    q           (time, latitude, longitude) float32 3MB 0.01255 ... 0.01351
    aermr11     (time, latitude, longitude) float32 3MB 2.595e-09 ... 3.006e-09
    so2         (time, latitude, longitude) float32 3MB 2.108e-09 ... 7.183e-10
    t           (time, latitude, longitude) float32 3MB 291.6 290.4 ... 296.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-17T19:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
merged = xr.merge([sfc, mlev])

/tmp/ipykernel_2205807/870353829.py:1: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  merged = xr.merge([sfc, mlev])
/tmp/ipykernel_2205807/870353829.py:1: FutureWarning: In a future version of xarray the default value for compat will change from compat='no_conflicts' to compat='override'. This is likely to lead to different results when combining overlapping variables with the same name. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set compat explicitly.
  merged = xr.merge([sfc, mlev])


In [7]:
merged

<xarray.Dataset> Size: 54MB
Dimensions:     (time: 8768, latitude: 9, longitude: 9)
Coordinates:
    number      int64 8B 0
  * time        (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
    step        timedelta64[ns] 8B 00:00:00
    surface     float64 8B 0.0
  * latitude    (latitude) float64 72B 36.0 35.25 34.5 33.75 ... 31.5 30.75 30.0
  * longitude   (longitude) float64 72B -86.0 -85.25 -84.5 ... -80.75 -80.0
    valid_time  (time) datetime64[ns] 70kB 2022-01-01 ... 2024-12-31T21:00:00
    hybrid      float64 8B 60.0
Data variables: (12/19)
    u10         (time, latitude, longitude) float32 3MB 0.1425 0.06673 ... 7.265
    v10         (time, latitude, longitude) float32 3MB 3.888 3.758 ... 3.548
    d2m         (time, latitude, longitude) float32 3MB 290.4 289.8 ... 293.6
    t2m         (time, latitude, longitude) float32 3MB 292.2 291.3 ... 297.6
    sp          (time, latitude, longitude) float32 3MB 9.749e+04 ... 1.011e+05
    pm2p5       (time, latitude, longitude) float32 3MB 6.126e-09 ... 7.598e-09
    ...          ...
    go3         (time, latitude, longitude) float32 3MB 4.276e-08 ... 6.73e-08
    c3h8        (time, latitude, longitude) float32 3MB 2.424e-10 ... 1.429e-10
    q           (time, latitude, longitude) float32 3MB 0.01255 ... 0.01351
    aermr11     (time, latitude, longitude) float32 3MB 2.595e-09 ... 3.006e-09
    so2         (time, latitude, longitude) float32 3MB 2.108e-09 ... 7.183e-10
    t           (time, latitude, longitude) float32 3MB 291.6 290.4 ... 296.0
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-12-17T19:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
print("Units in merged dataset:")
for var_name in merged.data_vars:
    units = merged[var_name].attrs.get('units', 'No units specified')
    print(f"{var_name}: {units}")

Units in merged dataset:
u10: m s**-1
v10: m s**-1
d2m: K
t2m: K
sp: Pa
pm2p5: kg m**-3
pm10: kg m**-3
c2h6: kg kg**-1
oh: kg kg**-1
c5h8: kg kg**-1
hno3: kg kg**-1
no2: kg kg**-1
no: kg kg**-1
go3: kg kg**-1
c3h8: kg kg**-1
q: kg kg**-1
aermr11: kg kg**-1
so2: kg kg**-1
t: K


In [9]:
# merged.to_netcdf('data/cams_merged.nc')

In [10]:
print(merged.data_vars)

Data variables:
    u10      (time, latitude, longitude) float32 3MB 0.1425 0.06673 ... 7.265
    v10      (time, latitude, longitude) float32 3MB 3.888 3.758 ... 3.338 3.548
    d2m      (time, latitude, longitude) float32 3MB 290.4 289.8 ... 292.2 293.6
    t2m      (time, latitude, longitude) float32 3MB 292.2 291.3 ... 296.1 297.6
    sp       (time, latitude, longitude) float32 3MB 9.749e+04 ... 1.011e+05
    pm2p5    (time, latitude, longitude) float32 3MB 6.126e-09 ... 7.598e-09
    pm10     (time, latitude, longitude) float32 3MB 8.772e-09 ... 1.16e-08
    c2h6     (time, latitude, longitude) float32 3MB 4.884e-10 ... 3.732e-10
    oh       (time, latitude, longitude) float32 3MB 2.922e-15 ... 9.231e-14
    c5h8     (time, latitude, longitude) float32 3MB 1.819e-10 ... 9.002e-12
    hno3     (time, latitude, longitude) float32 3MB 3.915e-10 ... 2.14e-09
    no2      (time, latitude, longitude) float32 3MB 5.96e-09 ... 1.173e-09
    no       (time, latitude, longitude) float32 3

In [11]:
df = merged.to_dataframe().reset_index()
df = df.sort_values('time').set_index(['time'])

In [12]:
# Update units for PM 
df['pm10'] = df['pm10'] * 1e9  # Convert from kg/m3 to µg/m3
df['pm2p5'] = df['pm2p5'] * 1e9  # Convert from kg/m3 to µg/m3

rho = df['sp'] / (287.05 * df['t'] * (1 + 0.61 * df['q']))
df['aermr11'] = df['aermr11'] * rho * 1e6 # Convert from kg/kg to µg/m3

In [13]:
# Update units for gases
molar_masses = {
    'c2h6': 30.07,   # g/mol
    'oh': 17.01,     # g/mol
    'c5h8': 68.12,   # g/mol
    'hno3': 63.01,   # g/mol
    'no2': 46.01,    # g/mol
    'no': 30.01,     # g/mol
    'go3': 48.,      # g/mol
    'c3h8': 44.01,   # g/mol
    'so2': 64.07     # g/mol
}

mm_air = 28.97  # g/mol

for gas, mm_gas in molar_masses.items():
    df[gas] = df[gas] * (mm_gas / mm_air) * 1e9  # Convert from kg/kg to ppb

In [14]:
# Feature engineering 
df['U10'] = np.sqrt(df['u10']**2. + df['v10']**2.) # wind speed at 10m
df['nox'] = df['no'] + df['no2']
df['voc'] = df['c2h6'] + df['c3h8'] + df['c5h8']
df['rh'] = (np.exp((17.625 * df['d2m']) / (243.04 + df['d2m'])) / 
            np.exp((17.625 * df['t']) / (243.04 + df['t']))) * 100.0 # relativd humidity

In [15]:
df.drop(['number', 'step', 'surface', 'valid_time', 'hybrid'], axis=1, inplace=True)

In [16]:
df = df[(df['pm10'] > 0) & (df['pm2p5'] > 0) & (df['aermr11'] > 0)]

In [17]:
df

,latitude,longitude,u10,v10,d2m,t2m,sp,pm2p5,pm10,c2h6,...,go3,c3h8,q,aermr11,so2,t,U10,nox,voc,rh
time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,36.0,-86.00,0.142535,3.887923,290.409424,292.208984,97494.875,6.126495,8.772485,0.506937,...,70.840256,0.368230,0.012546,0.002999,4.662637,291.640137,3.890535,9.517194,1.302910,98.168762
2022-01-01 00:00:00,31.5,-83.00,1.099139,3.469222,292.645508,296.432129,100745.125,5.627800,8.655828,0.375387,...,74.765465,0.132310,0.015000,0.001326,1.075397,295.963379,3.639177,3.072565,3.082698,95.196884
2022-01-01 00:00:00,31.5,-83.75,1.265093,3.968246,292.885498,297.080078,100518.375,6.042342,9.486843,0.367594,...,77.697563,0.130685,0.015088,0.001503,1.351113,296.242920,4.165025,2.961969,2.209628,95.145714
2022-01-01 00:00:00,31.5,-84.50,1.492327,4.158431,293.339111,297.542236,100431.625,5.891934,9.521404,0.342963,...,80.249573,0.111997,0.015144,0.001419,1.297935,296.797607,4.418098,2.429996,1.779222,95.012497
2022-01-01 00:00:00,31.5,-85.25,1.657855,4.255477,293.912109,297.353271,100336.125,5.686356,9.356338,0.332400,...,83.562637,0.106452,0.015512,0.001334,1.458442,296.953613,4.567008,2.204145,1.581509,95.606117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-31 21:00:00,34.5,-81.50,5.304272,3.670624,284.097656,292.222656,98461.875,5.049964,7.299199,0.446499,...,116.761368,0.230500,0.008398,0.002305,2.501247,291.726685,6.450487,2.651462,0.910810,89.054039
2024-12-31 21:00:00,34.5,-82.25,6.341869,3.243011,282.782715,292.185059,97963.125,3.423908,4.917939,0.453088,...,121.235291,0.242680,0.007261,0.001982,3.486444,291.069702,7.122951,2.965955,0.905216,88.126549
2024-12-31 21:00:00,34.5,-83.00,6.677563,2.157074,281.260254,291.092041,96228.875,1.596685,2.281791,0.455544,...,123.996033,0.243645,0.006281,0.001087,3.852719,289.431030,7.017323,2.658428,0.882617,88.217117


In [18]:
features = ['U10', 'c2h6', 'oh', 'c5h8', 'hno3', 'no2', 'no','go3', 'c3h8', 'so2', 't', 'nox', 'voc', 'rh']
targets = ['pm2p5', 'pm10']

X = df[features]
y = df[targets]

In [19]:
total_samples = df.shape[0]
split_idx = int(total_samples * 0.7)
split_time = X.index[split_idx]

In [20]:
X_train = X.loc[:split_time]
X_test = X.loc[split_time:]
y_train = y.loc[:split_time]
y_test = y.loc[split_time:]

In [21]:
test_data = {
    'X_test': X_test,
    'y_test': y_test,
    'X_log_test': np.log1p(X_test),
    'y_log_test': np.log1p(y_test),
    'features': features,
    'targets': targets
}

joblib.dump(test_data, 'test_data.pkl')

['test_data.pkl']

In [22]:
ts_split = TimeSeriesSplit(n_splits=3)

In [23]:
param_grid = {
    "bootstrap": [True],
    "max_depth": [20],
    "max_features": ["sqrt"],  
    "min_samples_leaf": [1, 5],
    "min_samples_split": [2, 10],
    "n_estimators": [100, 500]
}

rf_model = RandomForestRegressor(random_state=22, n_jobs=-1)
rf_grid = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=ts_split,
    n_jobs=-1,
    scoring='r2',
    verbose=2
)

rf_grid.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time= 1.4min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time= 3.9min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time= 2.9min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time= 2.9min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time= 3.0min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 3.0min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=100; tota

,estimator,RandomForestR...ndom_state=22)
,param_grid,"{'bootstrap': [True], 'max_depth': [20], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 5], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,500


In [24]:
print(f"Best parameters: {rf_grid.best_params_}")
print(f"Best CV score: {rf_grid.best_score_}")

Best parameters: {'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Best CV score: 0.5829961504303324


In [25]:
best_model = rf_grid.best_estimator_

y_pred_test = best_model.predict(X_test)

targets = ['pm2p5', 'pm10']
for i, target in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred_test[:, i]))
    r2 = r2_score(y_test.iloc[:, i], y_pred_test[:, i])
    print(f"{target}: RMSE = {rmse}, R² = {r2}")

pm2p5: RMSE = 5.639772975132074, R² = 0.6767344929860339
pm10: RMSE = 7.599730272629387, R² = 0.688873898826063


In [26]:
joblib.dump(best_model, 'rf_best_model.pkl')

json.dump(rf_grid.best_params_, open('rf_best_params.json', 'w'))

In [27]:
# Rerun the Random Forest model with log transform
X_log = np.log1p(X)
y_log = np.log1p(y)

X_log_train = X_log.loc[:split_time]
X_log_test = X_log.loc[split_time:]
y_log_train = y_log.loc[:split_time]
y_log_test = y_log.loc[split_time:]

rf_grid_log = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=ts_split,
    n_jobs=-1,
    scoring='r2',
    verbose=2
)

rf_grid_log.fit(X_log_train, y_log_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time= 8.6min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time= 2.7min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time= 2.6min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=100; total time= 2.7min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time= 2.7min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=100; total time= 1.2min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=10, n_estimators=500; tot

,estimator,RandomForestR...ndom_state=22)
,param_grid,"{'bootstrap': [True], 'max_depth': [20], 'max_features': ['sqrt'], 'min_samples_leaf': [1, 5], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,500


In [28]:
print(f"Best parameters: {rf_grid_log.best_params_}")
print(f"Best CV score: {rf_grid_log.best_score_}")

Best parameters: {'bootstrap': True, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 500}
Best CV score: 0.41534348279632255


In [29]:
best_model_log = rf_grid_log.best_estimator_

# Predict on test set (compute to get numpy array)
y_log_pred_test = best_model_log.predict(X_log_test)

# Evaluate each target
targets = ['pm2p5', 'pm10']
for i, target in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_log_test.iloc[:, i], y_log_pred_test[:, i]))
    r2 = r2_score(y_log_test.iloc[:, i], y_log_pred_test[:, i])
    print(f"{target}: RMSE = {rmse}, R² = {r2}")

pm2p5: RMSE = 0.5135046927132493, R² = 0.49494169553397405
pm10: RMSE = 0.5441909000070243, R² = 0.4797783505731419


In [30]:
# Save best model for log transform
joblib.dump(best_model_log, 'rf_best_model_log.pkl')

# Save parameters of best model for log transform
json.dump(rf_grid_log.best_params_, open('rf_best_params_log.json', 'w'))

In [31]:
# Now try with XGBoost
X_train = X.loc[:split_time]
X_test = X.loc[split_time:]
y_train = y.loc[:split_time]
y_test = y.loc[split_time:]

xgb_model = XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    max_bin=256,
    random_state=12,
    n_jobs=-1
)

xgb_multi = MultiOutputRegressor(xgb_model, n_jobs=-1)

xgb_param_grid = {
    'estimator__n_estimators': [300, 500],
    'estimator__max_depth': [3, 4],
    'estimator__learning_rate': [0.05, 0.1],
    'estimator__subsample': [0.8],
    'estimator__colsample_bytree': [0.8],
    'estimator__min_child_weight': [1, 5],
    'estimator__gamma': [0, 0.1],
    'estimator__reg_lambda': [1, 5]
}

xgb_grid = GridSearchCV(
    estimator=xgb_multi,
    param_grid=xgb_param_grid,
    cv=ts_split,
    scoring='r2',
    n_jobs=-1,
    verbose=2
)

xgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=5, min_samples_split=2, n_estimators=500; total time=10.2min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time= 8.1min
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.05, estimator__max_depth=3, estimator__min_child_weight=1, estimator__n_estimators=300, estimator__reg_lambda=1, estimator__subsample=0.8; total time=   7.0s
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.05, estimator__max_depth=3, estimator__min_child_weight=5, estimator__n_estimators=500, estimator__reg_lambda=5, estimator__subsample=0.8; total time=  21.1s
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.1, estimator__max_depth=3, estimator__min_child_weight=1, estimator__n_estimators=5

,estimator,MultiOutputRe... n_jobs=-1)
,param_grid,"{'estimator__colsample_bytree': [0.8], 'estimator__gamma': [0, 0.1], 'estimator__learning_rate': [0.05, 0.1], 'estimator__max_depth': [3, 4], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,TimeSeriesSpl...est_size=None)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'reg:squarederror'


In [32]:
print(f"Best parameters: {xgb_grid.best_params_}")
print(f"Best CV score: {xgb_grid.best_score_}")

Best parameters: {'estimator__colsample_bytree': 0.8, 'estimator__gamma': 0, 'estimator__learning_rate': 0.1, 'estimator__max_depth': 4, 'estimator__min_child_weight': 5, 'estimator__n_estimators': 300, 'estimator__reg_lambda': 5, 'estimator__subsample': 0.8}
Best CV score: 0.5119524200757345


In [33]:
best_model_xgb = xgb_grid.best_estimator_

y_pred_test = best_model_xgb.predict(X_test)

targets = ['pm2p5', 'pm10']
for i, target in enumerate(targets):
    rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred_test[:, i]))
    r2 = r2_score(y_test.iloc[:, i], y_pred_test[:, i])
    print(f"{target}: RMSE = {rmse}, R² = {r2}")

pm2p5: RMSE = 6.008835644652245, R² = 0.6330416202545166
pm10: RMSE = 8.130778913170822, R² = 0.6438734531402588


In [34]:
joblib.dump(best_model_xgb, 'xgb_best_model_log.pkl')

json.dump(xgb_grid.best_params_, open('xgb_best_params_log.json', 'w'))

[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.05, estimator__max_depth=3, estimator__min_child_weight=5, estimator__n_estimators=300, estimator__reg_lambda=1, estimator__subsample=0.8; total time=  10.1s
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.05, estimator__max_depth=4, estimator__min_child_weight=1, estimator__n_estimators=500, estimator__reg_lambda=1, estimator__subsample=0.8; total time=  21.4s
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.1, estimator__max_depth=3, estimator__min_child_weight=5, estimator__n_estimators=300, estimator__reg_lambda=5, estimator__subsample=0.8; total time=   9.3s
[CV] END estimator__colsample_bytree=0.8, estimator__gamma=0, estimator__learning_rate=0.1, estimator__max_depth=4, estimator__min_child_weight=1, estimator__n_estimators=500, estimator__reg_lambda=1, estimator__subsample=0.8; total time=   8.3s
[CV] END estim